In [1]:
import pandas as pd
import numpy as np

In [2]:
def read_command(path):
    with open(path) as f:
        lines = f.readlines()
        
    com = []    
    count = 0
    for l in lines:
        if count == 0 and '# Example' in l:
            count += 1
        elif count == 1 and l == '#\n':
            count += 1
        elif count == 2 and '########################' in l:
            count += 1
        elif count ==3 and l == '\n':
            count += 1
        elif count == 4:
            com.append(l.strip())
            count = 0
            
    return com

# path = './EGPSR Examples.txt'
# com_lst = read_command(path)  
# print('len: ',len(com_lst))
# com_lst[:10]

#path_lst = ['./EGPSR Examples_{i}.txt' for i in range(1,11)]
def read_all(num_files):
    com_lst = []
    for i in range(1,num_files+1):
        path = './Examples/EGPSR Examples_' + str(i)+ '.txt'
        com_lst.extend(read_command(path))
    return com_lst

num_files = 10
com_lst = read_all(num_files)
print('len: ',len(com_lst))
print(com_lst[:10])

len:  100000
['Locate -B_find- Mary -B_find_per-, pick -B_take- up -I_take- the orange -B_take_obj- from the dining -i- table -B_take_sour-, and put -B_take- it -B_take_obj- on the bookcase -B_take_dest- please', 'Could you bring -B_take- me -B_take_dest- some drinks -B_take_obj- from the microwave -B_take_sour-', 'Robot please spot -B_find- Jennifer -B_find_per- at the entrance -B_find_dest-, retrieve -B_take- the paprika -B_take_obj- from the counter -B_take_sour-, and bring -B_take- it -B_take_obj- to Skyler -B_take_dest-', 'Please go -B_go- to the end -i- table -B_go_dest-, locate -B_find- Jennifer -B_find_per-, and escort -B_guide- her -B_guide_per-', 'Could you salute -B_greet- the woman -i- dressed -i- in -i- a -i- red -i- coat -i- -i- -B_greet_per- in the kitchen -B_greet_dest-', 'Robot please get acquainted with -B_know- Robert -B_find_per- at the entrance -B_find_dest-, then greet -B_greet- him -B_greet_per-', 'Could you pour -B_take- some grape -i- juice -B_take_obj- in a cu

In [3]:
import itertools

def improve_labels(com_lst):
    res = []
    for i,com in enumerate(com_lst):

        #-b_obj  -->  -B_obj: to fix a labelling error in the command generator
        com = [ele if 'b_' not in ele else ele.replace('b_','B_') for ele in com.split()]
        com = ['-'+ele if ele == 'B_guide_dest-' else ele for ele in com]


        if '-B_say-' not in com and '-i-' not in com:
            res.append(com)
            continue
        elif '-B_say-' in com:
            if '-I_say-' not in com:
                labelled = com
            else:
                b_idx = com.index('-B_say-') + 1
                e_idx = com.index('-I_say-') -1
                first = com[:b_idx]
                end = com[e_idx:]
                
                need_label = com[b_idx:e_idx]
                
                #rem all -i- in the 'say' cases
                need_label = [ele for ele in need_label if ele != '-i-']
                
                interleave = list(itertools.chain(*zip(need_label, ['-I_say-']*len(need_label))))
                labelled = first + interleave + end
                
                com = labelled.copy()

        #deal with -i-
        elif '-i-' in com: 
            string = ' '.join(com)
            while '-i- -i-' in string:
                string = string.replace('-i- -i-', '-i-')

            while '-i- -' in string:
                string = string.replace('-i- -','-')

            com = string.split()


            idx_lst = []
            for idx,ele in enumerate(com):
                if ele == '-i-':
                    if len(idx_lst) == 0: 
                        idx_lst.append([idx])
                    elif idx == idx_lst[-1][-1] + 2:
                        idx_lst[-1].append(idx)
                    else:
                        idx_lst.append([idx])
            # if i == 0:
            #     print(com)
            #     print(idx_lst)

            for lst in idx_lst:
                end_label = com[lst[-1]+2]
                com[lst[0]] = end_label.replace(',','')
                if 'B' in end_label:
                    com[lst[-1]+2] = com[lst[-1]+2].replace('B','I').replace(',','')

                    
                for j,idx in enumerate(lst):
                    if j == 0:
                        continue
                    else:
                        label = com[lst[-1]+2]

                        #remove the comma for the earlier tokens (with i) if has comma
                        if ',' in label:
                            label = label.replace(',','')

                        com[idx] = label

            labelled = com.copy()

        res.append(labelled)     
    return res

res = improve_labels(com_lst)

In [4]:
# x = 'skii'.replace('t','')
# print(x)
# res[3]
# [' '.join(ele) for ele in res[3]]
' '.join(res[5])

'Robot please get acquainted with -B_know- Robert -B_find_per- at the entrance -B_find_dest-, then greet -B_greet- him -B_greet_per-'

In [5]:
def generate_seq_out(res):
    seq_in = []
    seq_out = []
    comma_seq_out = []
    for i,com in enumerate(res):
        in_lst = []
        clean_out_lst = []
        comma_out_lst = []
        for ele in com:
#             print('ele; ',ele)
            if ele[0] != '-':
                in_lst.append(ele)
                clean_out_lst.append('O')
                comma_out_lst.append('O')
            else:
                clean_out_lst[-1] = ele.replace('-','',2)
                comma_out_lst[-1] = ele.replace('-','',2)
                if ',' in ele:
                    clean_out_lst[-1] = clean_out_lst[-1].replace(',','')

        # for j,lab in enumerate(comma_out_lst):
        #     if ',' in lab


        seq_in.append(in_lst)
        seq_out.append(clean_out_lst)
        comma_seq_out.append(comma_out_lst)
    return seq_in, seq_out, comma_seq_out

seq_in, seq_out,comma_seq_out = generate_seq_out(res)
comma_seq_out[0]

['B_find',
 'B_find_per,',
 'B_take',
 'I_take',
 'O',
 'B_take_obj',
 'O',
 'O',
 'B_take_sour',
 'I_take_sour',
 'O',
 'B_take',
 'B_take_obj',
 'O',
 'O',
 'B_take_dest',
 'O']

In [26]:
# [' '.join(ele) for ele in comma_seq_out]
def command_with_comma(seq_in,comma_seq_out):
    comma_seq_in = []
    for sample_idx in range(len(seq_in)):
        comma_command = []
        command = seq_in[sample_idx]
        label_with_comma = comma_seq_out[sample_idx]
        for token_idx in range(len(command)):
            if ',' in label_with_comma[token_idx]:
                command_token = command[token_idx] + ','
            else:
                command_token = command[token_idx]
            comma_command.append(command_token)

        comma_seq_in.append(comma_command)

    return comma_seq_in

comma_seq_in =  command_with_comma(seq_in,comma_seq_out)
print([' '.join(ele) for ele in comma_seq_in][0])
print([' '.join(ele) for ele in comma_seq_out][0])

Locate Mary, pick up the orange from the dining, table, and put it on the bookcase please
B_find B_find_per, B_take I_take O B_take_obj O O B_take_sour, I_take_sour, O B_take B_take_obj O O B_take_dest O


In [6]:
def rem_say(seq_in,seq_out,remove = True):
    if remove:
        say_idx = [i for i,labels in enumerate(seq_out) if 'B_say' in labels]
        seq_in = [seq for i,seq in enumerate(seq_in) if i not in  say_idx]
        seq_out = [seq for i,seq in enumerate(seq_out) if i not in  say_idx]
    return seq_in, seq_out

def rem_dups(com_lst, lab_lst, rem = True):
    com_lst = [' '.join(lst) for lst in com_lst]
    lab_lst = [' '.join(lst) for lst in lab_lst]
    
        
    com_lst, lab_lst = rem_say(com_lst,lab_lst,remove = rem)
    
    df = pd.DataFrame (com_lst, columns = ['command'])
    df['label'] = lab_lst
    df2 = df.drop_duplicates(subset=["command"], keep='first')
    return df2

df = rem_dups(seq_in,seq_out)
df

,command,label
0,Locate Mary pick up the orange from the dining...,B_find B_find_per B_take I_take O B_take_obj O...
1,Could you bring me some drinks from the microwave,O O B_take B_take_dest O B_take_obj O O B_take...
2,Robot please spot Jennifer at the entrance ret...,O O B_find B_find_per O O B_find_dest B_take O...
3,Please go to the end table locate Jennifer and...,O B_go O O B_go_dest I_go_dest B_find B_find_p...
4,Could you salute the woman dressed in a red co...,O O B_greet O B_greet_per I_greet_per I_greet_...
...,...,...
66188,Robot please get the orange from the sink loca...,O O B_take O B_take_obj O O B_take_sour B_find...
66190,Robot please place paprika on the desk get the...,O O B_take B_take_obj O O B_take_dest B_take O...
66191,Set sponge on the end table pinpoint Francis a...,B_take B_take_obj O O B_take_dest I_take_dest ...
66192,Spot Charlie in the corridor retrieve the oran...,B_find B_find_per O O B_find_dest B_take O B_t...


## Write intent files

In [7]:
def write_unique_intent(df):
    label = df['label'].tolist()
    label_lst = [l.split() for l in label]
    # unique_labels
    intents = [ele[2:]  for lst in label_lst for ele in lst  if ele.count('_')==1] #ele[2:] to remove the B_, I_
    #intents.extend(['PAD', 'O'])
    unique_intent = [*set(intents)]
    unique_intent = ['PAD','O'] + unique_intent

    with open('intent_label.txt','w') as f:
        for intent in unique_intent:
            f.write(intent)
            f.write('\n')
    return unique_intent

unique_intent = write_unique_intent(df)
unique_intent

['PAD', 'O', 'go', 'guide', 'find', 'know', 'greet', 'follow', 'take']

# Train Dev Test split

In [8]:
from sklearn.model_selection import train_test_split

def split(seq_in,seq_out):
    # split the train out of the rest
    X_train, X_dev_test, Y_train, Y_dev_test = train_test_split(seq_in, seq_out, test_size=0.25, random_state=42)

    #split the val and test
    X_dev, X_test, Y_dev, Y_test = train_test_split(X_dev_test, Y_dev_test, test_size=0.8, random_state=42)
    
    df_train = pd.DataFrame({'seq_in': X_train, 'seq_out': Y_train})
    df_dev = pd.DataFrame({'seq_in': X_dev, 'seq_out': Y_dev})
    df_test = pd.DataFrame({'seq_in': X_test, 'seq_out': Y_test})
    
    env_dict = {'train' : df_train,
               'dev' : df_dev,
               'test' : df_test}
    return env_dict

    
env_dict = split(df['command'].tolist(),df['label'].tolist())
for env,df in env_dict.items():
    print(env,'\n')
    display(df)
    print('------------------------------------------------------------------')

train 



,seq_in,seq_out
0,Could you offer something to drink to all the ...,O O B_take B_take_obj I_take_obj I_take_obj O ...
1,Pinpoint Francis accompany him to the bookcase...,B_find B_find_per B_guide B_guide_per O O B_gu...
2,Deliver to me the food at the top dining table,B_take O B_take_dest O B_take_obj O O B_take_s...
3,Hand me the cleaning stuff at the middle botto...,B_take B_take_dest O B_take_obj I_take_obj O O...
4,Could you locate Robin look for Alex at the en...,O O B_find B_find_per B_find I_find B_find_per...
...,...,...
32789,Get acquainted with Patricia at the entrance t...,O O B_know B_find_per O O B_find_dest O B_gree...
32790,Offer something to drink to all the people dre...,B_take B_take_obj I_take_obj I_take_obj O B_ta...
32791,Please take the paprika from the bookcase gras...,O B_take O B_take_obj O O B_take_sour B_take O...
32792,Could you come behind the person at the desk,O O B_follow I_follow O B_follow_per O O B_fol...


------------------------------------------------------------------
dev 



,seq_in,seq_out
0,Offer something to eat to all the children dre...,B_take B_take_obj I_take_obj I_take_obj O B_ta...
1,Enter to the living room look for Skyler and f...,B_go O O B_go_dest I_go_dest B_find I_find B_f...
2,Enter to the bedroom pinpoint Alex and locate ...,B_go O O B_go_dest B_find B_find_per O B_find ...
3,Leave a bowl on the storage table and a spoon ...,B_take O B_take_obj O O B_take_dest I_take_des...
4,Bring the orange to the man wearing a yellow s...,B_take O B_take_obj O O B_take_dest I_take_des...
...,...,...
2181,Could you please leave a dish on the storage t...,O O O B_take O B_take_obj O O B_take_dest I_ta...
2182,Navigate to the dining room locate the fruits ...,B_go O O B_go_dest I_go_dest B_find O B_find_o...
2183,Offer something to drink to all the people dre...,B_take B_take_obj I_take_obj I_take_obj O B_ta...
2184,Place a snacks on the bookcase spot Charlie at...,B_take O B_take_obj O O B_take_dest B_find B_f...


------------------------------------------------------------------
test 



,seq_in,seq_out
0,Look for the orange set apple on the sink and ...,B_find I_find O B_find_obj B_take B_take_obj O...
1,Locate Alex in the bedroom find Linda and retr...,B_find B_find_per O O B_find_dest B_find B_fin...
2,Pick up the apple from the bookcase set it to ...,B_take I_take O B_take_obj O O B_take_sour B_t...
3,Leave a bowl on the storage table and a knife ...,B_take O B_take_obj O O B_take_dest I_take_des...
4,Robot please take the person at the exit to th...,O O B_guide O B_guide_per O O B_guide_sour O O...
...,...,...
8741,Get the apple from the counter leave a fruits ...,B_take O B_take_obj O O B_take_sour B_take O B...
8742,Robot please find Francis in the dining room p...,O O B_find B_find_per O O B_find_dest I_find_d...
8743,Offer something to drink to all the men pointi...,B_take B_take_obj I_take_obj I_take_obj O B_ta...
8744,Robot please accompany the person at the sink ...,O O B_guide O B_guide_per O O B_guide_sour O O...


------------------------------------------------------------------


In [9]:
print(env_dict['train']['seq_in'][35])
env_dict['train']['seq_out'][35]

Take the smallest food from the counter to the end table


'B_take O B_take_obj I_take_obj O O B_take_sour O O B_take_dest I_take_dest'

In [11]:
def write(f_name,data_lst, space = True):
    with open(f_name, 'w') as f:
        for lst in data_lst:
            if space == True:
                com = ' '.join(lst)
            else:
                com = ''.join(lst)
            f.write(com)
            f.write('\n')


# def gen_files(env_name, data_df, unique_intent):
#     # get seq out
#     lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
#     #get seq in
#     com_lst = [ele.split() for ele in data_df['seq_in'].tolist()]
#
#     # get intent out
#     int_lst = []
#     for sample_lab in lab_lst:
#         lst = []
#         for ele in sample_lab:
#             if '_' not in ele:
#                 lst.append('O')
#             else:
#                 start = ele.find('_') + 1
#                 if ele.count('_') == 2:
#                     end = ele.rfind('_')
#                     lst.append(ele[start:end])
# #################### turns out we don't need to label the verb ###################
#                 else:
#                     lst.append('O') # verb -> 'O'
# #                     lst.append(ele[start:])
#
#
#         int_lst.append(lst)
#
#     # get label
#     label = []
#     for lab_seq in int_lst:
#         lst = []
#         for lab in lab_seq:
#             if lab == 'O':
#                 continue
#             elif len(lst) == 0:
#                 lst.append(lab)
#             elif lab != lst[-1]:
#                 lst.append(lab)
#         hash_taged = [ele+'#' if i != len(lst)-1 else ele for i,ele in enumerate(lst)]
#         label.append(hash_taged)
#
#
#     # get seq out
#     # 1. remove the intent label out of the seq out
#     lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
#     lab_lst = [[ele if ele.count('_') == 2 else 'O' for ele in lst] for lst in lab_lst]
#     # 2. remove the intentions names in the slot names
#     slot_seq = []
#     for ite,one_seq in enumerate(lab_lst):
#         lst = []
#         for slot in one_seq:
#
#             if slot == 'O':
#                 lst.append('O')
#             else:
#                 for uni_inte in unique_intent: #ite through all possible uni slots
#                     idx1 = slot.find(uni_inte) # find whether this uni slot is in current seq slot labels
#                     if idx1 != -1:
#                         s = slot.replace(slot[idx1 : idx1+len(uni_inte)+1],'')
#                         s = s.replace('_','-')
#                         lst.append(s)
#                         continue
#             #print(lst)
#
#         slot_seq.append(lst)
#
#     df = data_df.copy()
#     df['seq_out'] = slot_seq
#     df['seq_intent'] = int_lst
#     df['label'] = label
#
#
#     # write all files
#     write(f'./{env_name}/seq.in',com_lst)
#     write(f'./{env_name}/seq.out',slot_seq)
#     write(f'./{env_name}/seq_intent.out',int_lst)
#     write(f'./{env_name}/label',label, False)
#
#     return df












def gen_files(env_name, data_df, unique_intent):
    # get seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
    #get seq in
    com_lst = [ele.split() for ele in data_df['seq_in'].tolist()]

    # get intent out
    int_lst = []
    for sample_lab in lab_lst:
        lst = []
        for ele in sample_lab:
            if '_' not in ele:
                lst.append('O')
            else:
                start = ele.find('_') + 1
                if ele.count('_') == 2:
                    end = ele.rfind('_')
                    lst.append(ele[start:end])
#################### turns out we don't need to label the verb ###################
                else:
                    lst.append('O') # verb -> 'O'
#                     lst.append(ele[start:])


        int_lst.append(lst)

    # get label
    label = []
    for lab_seq in int_lst:
        lst = []
        for lab in lab_seq:
            if lab == 'O':
                continue
            elif len(lst) == 0:
                lst.append(lab)
            elif lab != lst[-1]:
                lst.append(lab)
        hash_taged = [ele+'#' if i != len(lst)-1 else ele for i,ele in enumerate(lst)]
        label.append(hash_taged)


    # get seq out
    # 1. remove the intent label out of the seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
    lab_lst = [[ele if ele.count('_') == 2 else 'O' for ele in lst] for lst in lab_lst]
    # 2. remove the intentions names in the slot names
    slot_seq = []
    for ite,one_seq in enumerate(lab_lst):
        lst = []
        for slot in one_seq:

            if slot == 'O':
                lst.append('O')
            else:
                # with IBO
                slot_start = slot.find('_',2)+1
                entity_type = slot[slot_start:]
                lst.append(slot[0]+'-'+entity_type)

                #without IBO
                # slot_start = slot.find('_',2)+1
                # lst.append(slot[slot_start:])

        slot_seq.append(lst)

    df = data_df.copy()
    df['seq_out'] = slot_seq
    df['seq_intent'] = int_lst
    df['label'] = label


    # write all files
    write(f'./{env_name}/seq.in',com_lst)
    write(f'./{env_name}/seq.out',slot_seq)
    write(f'./{env_name}/seq_intent.out',int_lst)
    write(f'./{env_name}/label',label, False)

    return df



df_train_all = gen_files('train',env_dict['train'], unique_intent)
df_dev_all = gen_files('dev',env_dict['dev'], unique_intent)
df_test_all = gen_files('test',env_dict['test'], unique_intent)

env_dict_all = {'train' : df_train_all,
               'dev' : df_dev_all,
               'test' : df_test_all}

for env,df in env_dict_all.items():
    print(env,'\n')
    display(df)
    print('------------------------------------------------------------------')

train 



,seq_in,seq_out,seq_intent,label
0,Could you offer something to drink to all the ...,"[O, O, O, B-obj, I-obj, I-obj, O, B-dest, I-de...","[O, O, O, take, take, take, O, take, take, tak...",[take]
1,Pinpoint Francis accompany him to the bookcase...,"[O, B-per, O, B-per, O, O, B-dest, O, O, O, B-...","[O, find, O, guide, O, O, guide, O, O, O, take...","[find#, guide#, take]"
2,Deliver to me the food at the top dining table,"[O, O, B-dest, O, B-obj, O, O, B-sour, I-sour,...","[O, O, take, O, take, O, O, take, take, take]",[take]
3,Hand me the cleaning stuff at the middle botto...,"[O, B-dest, O, B-obj, I-obj, O, O, B-sour, I-s...","[O, take, O, take, take, O, O, take, take, tak...",[take]
4,Could you locate Robin look for Alex at the en...,"[O, O, O, B-per, O, O, B-per, O, O, B-dest, I-...","[O, O, O, find, O, O, find, O, O, find, find, ...","[find#, guide]"
...,...,...,...,...
32789,Get acquainted with Patricia at the entrance t...,"[O, O, O, B-per, O, O, B-dest, O, O, B-per, O]","[O, O, O, find, O, O, find, O, O, greet, O]","[find#, greet]"
32790,Offer something to drink to all the people dre...,"[O, B-obj, I-obj, I-obj, O, B-dest, I-dest, I-...","[O, take, take, take, O, take, take, take, tak...",[take]
32791,Please take the paprika from the bookcase gras...,"[O, O, O, B-obj, O, O, B-sour, O, O, B-obj, O,...","[O, O, O, take, O, O, take, O, O, take, O, O, ...",[take]
32792,Could you come behind the person at the desk,"[O, O, O, O, O, B-per, O, O, B-sour]","[O, O, O, O, O, follow, O, O, follow]",[follow]


------------------------------------------------------------------
dev 



,seq_in,seq_out,seq_intent,label
0,Offer something to eat to all the children dre...,"[O, B-obj, I-obj, I-obj, O, B-dest, I-dest, I-...","[O, take, take, take, O, take, take, take, tak...",[take]
1,Enter to the living room look for Skyler and f...,"[O, O, O, B-dest, I-dest, O, O, B-per, O, O, B...","[O, O, O, go, go, O, O, find, O, O, follow]","[go#, find#, follow]"
2,Enter to the bedroom pinpoint Alex and locate ...,"[O, O, O, B-dest, O, B-per, O, O, B-per, O, O,...","[O, O, O, go, O, find, O, O, find, O, O, find,...","[go#, find]"
3,Leave a bowl on the storage table and a spoon ...,"[O, O, B-obj, O, O, B-dest, I-dest, O, O, B-ob...","[O, O, take, O, O, take, take, O, O, take, O, ...",[take]
4,Bring the orange to the man wearing a yellow s...,"[O, O, B-obj, O, O, B-dest, I-dest, I-dest, I-...","[O, O, take, O, O, take, take, take, take, tak...",[take]
...,...,...,...,...
2181,Could you please leave a dish on the storage t...,"[O, O, O, O, O, B-obj, O, O, B-dest, I-dest, O...","[O, O, O, O, O, take, O, O, take, take, O, O, ...",[take]
2182,Navigate to the dining room locate the fruits ...,"[O, O, O, B-dest, I-dest, O, O, B-obj, O, O, O...","[O, O, O, go, go, O, O, find, O, O, O, take, O...","[go#, find#, take]"
2183,Offer something to drink to all the people dre...,"[O, B-obj, I-obj, I-obj, O, B-dest, I-dest, I-...","[O, take, take, take, O, take, take, take, tak...",[take]
2184,Place a snacks on the bookcase spot Charlie at...,"[O, O, B-obj, O, O, B-dest, O, B-per, O, O, B-...","[O, O, take, O, O, take, O, find, O, O, find, ...","[take#, find]"


------------------------------------------------------------------
test 



,seq_in,seq_out,seq_intent,label
0,Look for the orange set apple on the sink and ...,"[O, O, O, B-obj, O, B-obj, O, O, B-dest, O, O,...","[O, O, O, find, O, take, O, O, take, O, O, O, ...","[find#, take]"
1,Locate Alex in the bedroom find Linda and retr...,"[O, B-per, O, O, B-dest, O, B-per, O, O, O, B-...","[O, find, O, O, find, O, find, O, O, O, take, ...","[find#, take]"
2,Pick up the apple from the bookcase set it to ...,"[O, O, O, B-obj, O, O, B-sour, O, B-obj, O, O,...","[O, O, O, take, O, O, take, O, take, O, O, tak...","[take#, find]"
3,Leave a bowl on the storage table and a knife ...,"[O, O, B-obj, O, O, B-dest, I-dest, O, O, B-ob...","[O, O, take, O, O, take, take, O, O, take, O, ...",[take]
4,Robot please take the person at the exit to th...,"[O, O, O, O, B-per, O, O, B-sour, O, O, B-dest...","[O, O, O, O, guide, O, O, guide, O, O, guide, ...",[guide]
...,...,...,...,...
8741,Get the apple from the counter leave a fruits ...,"[O, O, B-obj, O, O, B-sour, O, O, B-obj, O, O,...","[O, O, take, O, O, take, O, O, take, O, O, tak...","[take#, find]"
8742,Robot please find Francis in the dining room p...,"[O, O, O, B-per, O, O, B-dest, I-dest, O, O, B...","[O, O, O, find, O, O, find, find, O, O, take, ...","[find#, take]"
8743,Offer something to drink to all the men pointi...,"[O, B-obj, I-obj, I-obj, O, B-dest, I-dest, I-...","[O, take, take, take, O, take, take, take, tak...",[take]
8744,Robot please accompany the person at the sink ...,"[O, O, O, O, B-per, O, O, B-sour, O, O, B-dest...","[O, O, O, O, guide, O, O, guide, O, O, guide, ...",[guide]


------------------------------------------------------------------


In [12]:
print(env_dict_all['train']['seq_out'][0])
print(env_dict_all['train']['seq_intent'][0])

['O', 'O', 'O', 'B-obj', 'I-obj', 'I-obj', 'O', 'B-dest', 'I-dest', 'I-dest', 'I-dest', 'I-dest', 'O', 'O', 'B-dest', 'I-dest']
['O', 'O', 'O', 'take', 'take', 'take', 'O', 'take', 'take', 'take', 'take', 'take', 'O', 'O', 'take', 'take']


# Write unique slots files

In [13]:
def write_unique_slot(env_dict_all):
    label = []
    for df in env_dict_all.values():
        label.extend(df['seq_out'].tolist())
        
    label_lst = label.copy()
    # unique_labels
#     ites = [i for i,lst in enumerate(label_lst) if 'b-obj' in lst]
#     print(ites[:10])
#     print(df['seq_in'][ites[0]])
#     print(label[ites[0]])
    
    slots = [ele  for lst in label_lst for ele in lst if ele != 'O'] #ele[2:] to remove the B_, I_
    unique_slot = ['PAD','O'] + [*set(slots)]
    
    with open('slot_label.txt','w') as f:
        for slot in unique_slot:
            f.write(slot)
            f.write('\n')
    return unique_slot

unique_slot = write_unique_slot(env_dict_all)
unique_slot

['PAD',
 'O',
 'B-obj',
 'B-dest',
 'B-per',
 'B-sour',
 'I-dest',
 'I-sour',
 'I-obj',
 'I-per']

# Check labels dist, and write unique labels (+ PAD, UNK, but UNK is not going to be used)

In [14]:
# def unique_labels(file_path):
#     labels = [line.strip().split() for line in open(file_path, 'r', encoding='utf-8')]
#     label_df = pd.Series( x for seq in labels for x in seq)
#     lst = label_df.unique().tolist()
#     return lst

def show_unique_labels(env):
    uni_slot = [line.strip() for line in open('slot_label.txt', 'r', encoding='utf-8')]
    uni_intent = [line.strip() for line in open('intent_label.txt', 'r', encoding='utf-8')]
    
    slot_path =  f'./{env}/seq.out'
    intent_path = f'./{env}/seq_intent.out'
    
    slot_seq = [line.strip().split() for line in open(slot_path, 'r', encoding='utf-8')]
    intent_seq = [line.strip().split() for line in open(intent_path, 'r', encoding='utf-8')]
    
    slot_lst = [label for seq in slot_seq for label in seq]
    intent_lst = [label for seq in intent_seq for label in seq] 
    
    env_uni_slot = [*set(slot_lst)]
    env_uni_intent = [*set(intent_lst)]
    print('env: ',env)
    print('Unique intent: ',env_uni_intent,len(env_uni_intent))
    print('Unique slots:  ',env_uni_slot,len(env_uni_slot),'\n')
    return


for env in env_dict_all.keys():
    show_unique_labels(env)

env:  train
Unique intent:  ['go', 'guide', 'find', 'O', 'greet', 'follow', 'take'] 7
Unique slots:   ['B-obj', 'B-dest', 'B-per', 'B-sour', 'I-dest', 'I-sour', 'O', 'I-obj', 'I-per'] 9 

env:  dev
Unique intent:  ['go', 'guide', 'find', 'O', 'greet', 'follow', 'take'] 7
Unique slots:   ['B-obj', 'B-dest', 'B-per', 'B-sour', 'I-dest', 'I-sour', 'O', 'I-obj', 'I-per'] 9 

env:  test
Unique intent:  ['go', 'guide', 'find', 'O', 'greet', 'follow', 'take'] 7
Unique slots:   ['B-obj', 'B-dest', 'B-per', 'B-sour', 'I-dest', 'I-sour', 'O', 'I-obj', 'I-per'] 9 

